# Modul 8: Deep Learning and NLP

## **Setup**

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
import pathlib

2021-09-13 19:18:09.777406: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-13 19:18:09.777717: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Preload word embeddings


In [13]:
# Load pre-trained word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-09-13 18:41:53--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-09-13 18:41:54--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-09-13 18:41:54--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [22]:
!unzip glove.6B.zip.1 -d glove.6B

Archive:  glove.6B.zip.1
  inflating: glove.6B.txt/glove.6B.50d.txt  
  inflating: glove.6B.txt/glove.6B.100d.txt  
  inflating: glove.6B.txt/glove.6B.200d.txt  
  inflating: glove.6B.txt/glove.6B.300d.txt  


### **The data**

In [2]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

In [3]:
data_dir = pathlib.Path(data_path).parent / "20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of directories: 20
Directory names: ['sci.crypt', 'misc.forsale', 'sci.med', 'rec.sport.hockey', 'alt.atheism', 'comp.sys.mac.hardware', 'comp.os.ms-windows.misc', 'talk.politics.mideast', 'soc.religion.christian', 'talk.politics.misc', 'talk.politics.guns', 'rec.motorcycles', 'comp.windows.x', 'comp.graphics', 'rec.sport.baseball', 'comp.sys.ibm.pc.hardware', 'sci.electronics', 'sci.space', 'rec.autos', 'talk.religion.misc']
Number of files in comp.graphics: 1000
Some example filenames: ['38253', '38469', '39656', '38291', '38334']


In [4]:
# Get an example of what the dataset contains
print(open(data_dir / "comp.graphics" / "38987").read())

Newsgroups: comp.graphics
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!howland.reston.ans.net!agate!dog.ee.lbl.gov!network.ucsd.edu!usc!rpi!nason110.its.rpi.edu!mabusj
From: mabusj@nason110.its.rpi.edu (Jasen M. Mabus)
Subject: Looking for Brain in CAD
Message-ID: <c285m+p@rpi.edu>
Nntp-Posting-Host: nason110.its.rpi.edu
Reply-To: mabusj@rpi.edu
Organization: Rensselaer Polytechnic Institute, Troy, NY.
Date: Thu, 29 Apr 1993 23:27:20 GMT
Lines: 7

Jasen Mabus
RPI student

	I am looking for a hman brain in any CAD (.dxf,.cad,.iges,.cgm,etc.) or picture (.gif,.jpg,.ras,etc.) format for an animation demonstration. If any has or knows of a location please reply by e-mail to mabusj@rpi.edu.

Thank you in advance,
Jasen Mabus  



### Preprocessing

In [5]:
# Unpack and load the data
samples = []
labels = []
class_names = []
class_index = 0
for dirname in sorted(os.listdir(data_dir)):
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.os.ms-windows.misc, 1000 files found
Processing comp.sys.ibm.pc.hardware, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.motorcycles, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.electronics, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.mideast, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.ha

In [6]:
# Create a training and validation dataset

# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

## **Exercise 8.1: Vectorization**

In [7]:
from tensorflow.keras.layers import TextVectorization

text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
# Create a TextVectorization with 20000 max_tokens and an output_sequence_length of 200
# Adapt the vectorizer to the data
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
vectorizer.adapt(text_ds)

2021-09-13 19:18:18.890938: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-13 19:18:18.891495: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-13 19:18:18.891589: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (CE43859): /proc/driver/nvidia/version does not exist
2021-09-13 19:18:18.894136: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-13 19:18:20.159478: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimi

In [8]:
# We can have a look at the vocabulary index
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [9]:
# If we vectorize a sentence we get
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3664, 1805,   15,    2, 8148])

## **Exercise 8.2: Embeddings**

In [17]:
# Extract the installed word embeddings
# We use the smallest available embedding 50
path_to_glove_file = 'glove.6B/glove.6B.50d.txt'

# Every entry in the file contains the word followed by the coefficients
# Extract all the words into a dictionary with key (word) value(vector) mapping 
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


The embedding is a simple NumPy matrix where entry at index i is the pre-trained vector for the word of index i in our vectorizer's vocabulary.

In [18]:
# Create an embedding matrix to send it to the embedding layer later

# Create another mapping for the vocabulary learned by the vectorizer
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

num_tokens = len(voc) + 2
embedding_dim = 50
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
# Join both dictionaries in the numpy array
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 17956 words (2044 misses)


In [19]:
# Create the embedding layer
# We pass our embedding matrix as the state of the matrix and set it to not trainable
embedding_layer = layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

## **Exercise 8.3: Modeling**

In [20]:
# Build a convoluational model that takes the vectorized words as input

int_sequences_input = keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = layers.Conv1D(128, 5, activation="relu")(embedded_sequences)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(128, 5, activation="relu")(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          1000100   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         32128     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         8204

In [21]:
# Use right padding to make each sentence the same size
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [22]:
# Compile the model
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)

In [23]:
# Fit the model
# With 20 epochs this should take about 3 minutes
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 12s 74ms/step - loss: 2.6071 - acc: 0.1558 - val_loss: 2.1245 - val_acc: 0.2748
Epoch 2/20
125/125 [==============================] - 11s 90ms/step - loss: 1.9266 - acc: 0.3320 - val_loss: 1.5857 - val_acc: 0.4554
Epoch 3/20
125/125 [==============================] - 9s 74ms/step - loss: 1.5967 - acc: 0.4401 - val_loss: 1.3659 - val_acc: 0.5396
Epoch 4/20
125/125 [==============================] - 9s 74ms/step - loss: 1.3808 - acc: 0.5256 - val_loss: 1.3038 - val_acc: 0.5596
Epoch 5/20
125/125 [==============================] - 10s 83ms/step - loss: 1.2336 - acc: 0.5791 - val_loss: 1.1625 - val_acc: 0.6032
Epoch 6/20
125/125 [==============================] - 10s 80ms/step - loss: 1.0940 - acc: 0.6205 - val_loss: 1.1549 - val_acc: 0.6084
Epoch 7/20
125/125 [==============================] - 10s 78ms/step - loss: 0.9865 - acc: 0.6630 - val_loss: 1.0716 - val_acc: 0.6422
Epoch 8/20
125/125 [==============================] - 10s 78ms/s

## **Bonus: Query the model**

In [24]:
# Create an end-to-end model
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

In [27]:
sentence = [['Put your text here']]

In [28]:
probabilities = end_to_end_model.predict(sentence)

class_names[np.argmax(probabilities[0])]

'talk.politics.guns'

## **Exercise 8.4: Recurrent Models**

In [29]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = embedding_layer = layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
preds = layers.Dense(len(class_names), activation="softmax")(x)
model = keras.Model(inputs, preds)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 50)          1000100   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         58880     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_4 (Dense)              (None, 20)                2580      
Total params: 1,160,376
Trainable params: 160,276
Non-trainable params: 1,000,100
_________________________________________________________________


In [30]:
# Compule the model
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)

In [31]:
# Fit the model
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))

Epoch 1/20
125/125 [==============================] - 89s 681ms/step - loss: 2.3852 - acc: 0.2272 - val_loss: 2.1526 - val_acc: 0.2663
Epoch 2/20
125/125 [==============================] - 82s 656ms/step - loss: 1.9227 - acc: 0.3512 - val_loss: 1.8041 - val_acc: 0.3598
Epoch 3/20
125/125 [==============================] - 82s 654ms/step - loss: 1.7385 - acc: 0.4056 - val_loss: 1.5671 - val_acc: 0.4564
Epoch 4/20
125/125 [==============================] - 82s 656ms/step - loss: 1.6107 - acc: 0.4424 - val_loss: 1.7591 - val_acc: 0.3773
Epoch 5/20
125/125 [==============================] - 79s 635ms/step - loss: 1.5388 - acc: 0.4714 - val_loss: 1.4550 - val_acc: 0.4964
Epoch 6/20
125/125 [==============================] - 81s 648ms/step - loss: 1.4351 - acc: 0.5070 - val_loss: 1.4258 - val_acc: 0.5101
Epoch 7/20
125/125 [==============================] - 85s 679ms/step - loss: 1.3685 - acc: 0.5313 - val_loss: 1.3516 - val_acc: 0.5314
Epoch 8/20
125/125 [==============================] - 7

## **Bonus: Query the model**

In [32]:
# Create an end-to-end model
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = model(x)
end_to_end_model = keras.Model(string_input, preds)

In [35]:
sentence = [['Put your text here']]

In [36]:
probabilities = end_to_end_model.predict(sentence)

class_names[np.argmax(probabilities[0])]

'talk.politics.mideast'